# Lab 2

In [10]:
from mable.cargo_bidding import TradingCompany
from mable.transport_operation import Schedule, ScheduleProposal, Bid

class MyCompany(TradingCompany):
    
    def pre_inform(self, trades, time):
        pass

    def inform(self, trades, *args, **kwargs):
        bids = []
        for trade in trades:
            for vessel in self.fleet:
                cost = self.estimate_cost(vessel, trade)
                bid_amount = cost * 1.1  # Add margin
                bid = Bid(trade, bid_amount)
                bids.append(bid)
                break

        return bids

    def receive(self, contracts, auction_ledger, *args, **kwargs):
        pass

    def propose_schedules(self, trades):
        return self.create_schedules_for_trades(trades)

    def create_schedules_for_trades(self, trades):
        schedules = []
        costs = []

        for trade in trades:
            for vessel in self.fleet:
                schedule = Schedule(vessel)
                schedule.add_transportation(trade)

                if schedule.verify_schedule():
                    cost = self.estimate_cost(vessel, trade)
                    schedules.append(schedule)
                    costs.append(cost)
                    break

        return ScheduleProposal(trades, schedules, costs)

    def estimate_cost(self, vessel, trade):
        # Use `self.headquarters.get_network_distance` for distance calculation
        distance = self.headquarters.get_network_distance(trade.origin_port, trade.destination_port)

        loading_cost = vessel.get_loading_consumption() * vessel.fuel_price
        travel_cost = vessel.get_laden_consumption(distance) * vessel.fuel_price
        unloading_cost = vessel.get_unloading_consumption() * vessel.fuel_price

        total_cost = loading_cost + travel_cost + unloading_cost
        return total_cost


In [ ]:
# Assuming the simulation environment is set up
my_company = MyCompany()

# Example market interactions
trades = market.get_upcoming_trades()
time = market.get_time()

# Receive advance information about future trades
my_company.pre_inform(trades, time)

# Participate in current auction with scheduling and bidding
bids = my_company.inform(trades)
market.submit_bids(bids)

# Process auction results
contracts_won, auction_ledger = market.get_results()
my_company.receive(contracts_won, auction_ledger)
